# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json 

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Klyuchi,52.2667,79.1667,-1.37,68,100,4.41,RU,1699297927
1,1,College,64.8569,-147.8028,-7.03,88,100,0.00,US,1699297927
2,2,Lagoa Vermelha,-28.2086,-51.5258,24.61,56,0,0.39,BR,1699297928
3,3,Adamstown,-25.0660,-130.1015,22.95,78,100,7.04,PN,1699297928
4,4,Yung Shue Wan,22.2333,114.1167,26.46,58,12,3.74,HK,1699297928


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    size = "Humidity",
    color = 'City',
)
# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
city_weather_df = city_data_df[(city_data_df['Max Temp'] >= 29) & (city_data_df['Max Temp'] < 30)]

# Drop any rows with null values
city_weather_df.dropna(how='any')

# Display sample data
city_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
84,84,San Luis de la Loma,17.2706,-100.8939,29.86,70,95,2.13,MX,1699297946
142,142,Homosassa Springs,28.8036,-82.5759,29.57,53,0,4.63,US,1699297959
153,153,Mutuáli,-14.8706,37.0044,29.74,37,100,0.84,MZ,1699297961
173,173,Trairi,-3.2778,-39.2689,29.21,58,13,6.16,BR,1699297938
216,216,Lázaro Cárdenas,17.9583,-102.2000,29.96,67,11,3.13,MX,1699297755
238,238,Farasān,16.7022,42.1183,29.82,69,100,1.72,SA,1699297976
260,260,Burumburum,11.3911,8.7234,29.33,19,100,3.22,NG,1699297980
272,272,Salinópolis,-0.6136,-47.3561,29.28,72,57,5.89,BR,1699297972
421,421,Marco Island,25.9363,-81.7157,29.00,41,0,2.57,US,1699298017
446,446,Acaraú,-2.8856,-40.1200,29.36,61,13,8.31,BR,1699298023


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_weather_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
84,San Luis de la Loma,MX,17.2706,-100.8939,70,
142,Homosassa Springs,US,28.8036,-82.5759,53,
153,Mutuáli,MZ,-14.8706,37.0044,37,
173,Trairi,BR,-3.2778,-39.2689,58,
216,Lázaro Cárdenas,MX,17.9583,-102.2000,67,
238,Farasān,SA,16.7022,42.1183,69,
260,Burumburum,NG,11.3911,8.7234,19,
272,Salinópolis,BR,-0.6136,-47.3561,72,
421,Marco Island,US,25.9363,-81.7157,41,
446,Acaraú,BR,-2.8856,-40.1200,61,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Longitude = hotel_df.loc[index, "Lng"]
    Latitude = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{Longitude},{Latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    #name_address = 
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
San Luis de la Loma - nearest hotel: No hotel found
Homosassa Springs - nearest hotel: Bella Oasis Hotel
Mutuáli - nearest hotel: No hotel found
Trairi - nearest hotel: Pousada Villa Aurora
Lázaro Cárdenas - nearest hotel: Hotel Sol del Pacífico
Farasān - nearest hotel: No hotel found
Burumburum - nearest hotel: No hotel found
Salinópolis - nearest hotel: Hotel Salinópolis
Marco Island - nearest hotel: JW Marriott Marco Island Beach Resort
Acaraú - nearest hotel: No hotel found
Mā‘ili - nearest hotel: No hotel found
Ferkessédougou - nearest hotel: Ferké Hôtel Iroko
Palmas - nearest hotel: Hotel Triângulo Mineiro


,City,Country,Lat,Lng,Humidity,Hotel Name
84,San Luis de la Loma,MX,17.2706,-100.8939,70,No hotel found
142,Homosassa Springs,US,28.8036,-82.5759,53,Bella Oasis Hotel
153,Mutuáli,MZ,-14.8706,37.0044,37,No hotel found
173,Trairi,BR,-3.2778,-39.2689,58,Pousada Villa Aurora
216,Lázaro Cárdenas,MX,17.9583,-102.2000,67,Hotel Sol del Pacífico
238,Farasān,SA,16.7022,42.1183,69,No hotel found
260,Burumburum,NG,11.3911,8.7234,19,No hotel found
272,Salinópolis,BR,-0.6136,-47.3561,72,Hotel Salinópolis
421,Marco Island,US,25.9363,-81.7157,41,JW Marriott Marco Island Beach Resort
446,Acaraú,BR,-2.8856,-40.1200,61,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ['Lng','Lat','City','Humidity','Hotel Name','Country']
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)